## A preliminary view at 2020 mortality data from ISTAT

You can find Ricciato's original study on [CROS portal](https://ec.europa.eu/eurostat/cros/content/preliminary-view-2020-mortality-data-istat).

### Setup of the environment

In [35]:
import os, sys, io
import requests
try:
    import numpy as np
    import pandas as pd
except:
    print('Critical packages missing... abort!')
try:
    import zipfile
except:
    print('zip facility missing...')

### Data ingestion

Actually at this stage, we set manually the metadata. 

In [4]:
# next: we will use pandoc to load information regarding the metadata from docx

In [94]:
from collections import OrderedDict
from datetime import datetime
OBASETYPE       = {t.__name__: t for t in [type, bool, int, float, str, datetime]}
__type2name     = lambda t: t.__name__  # lambda t: {v:k for (k,v) in OBASETYPE.items()}[t]    

METADATA        =  { "country":     {'code': 'IT', 'name': 'Italia'},
                     "lang":        {'code': 'it', 'name': 'italian'}, 
                     "file":        'comune_giorno.csv',
                     "source":      'https://www.istat.it/it/files/2020/03/comune-giorno.zip',
                     "enc":         "latin1",
                     "sep":         ';', 
                     "date":        None, #'%d-%m-%Y %H:%M',
                     "index":     OrderedDict( [ 
                        ("REG",       {"name": "REG",                     "desc": "Codice Istat della Regione di residenza.",
                                      "type": __type2name(int),         "values": None}),
                        ("PROV",     {"name": "PROV",          "desc": "Codice Istat della Provincia di residenza.",   
                                      "type": __type2name(int),         "values": None}),
                        ("REGIONE",     {"name": "NOME_REGIONE",          "desc": "Regione di residenza.",   
                                      "type": __type2name(str),         "values": None}),
                        ("PROVINCIA",     {"name": "NOME_PROVINCIA",          "desc": "Provincia di residenza.",   
                                      "type": __type2name(str),         "values": None}),
                        ("COMUNE",     {"name": "NOME_COMUNE",          "desc": "Comune di residenza.",   
                                      "type": __type2name(str),         "values": None}),
                        ("PROVCOM",     {"name": "COD_PROVCOM",          "desc": "Comune di residenza (classificazione Istat al 01/01/2020)",   
                                      "type": __type2name(str),         "values": None}),
                        ("ETA",     {"name": "CL_ETA",          "desc": "Classe di età in anni compiuti al momento del decesso",   
                                      "type": __type2name(int),         "values": {0: "0", 1: "1-4", 2: "5-9", 3:"10-14", 4:"15-19", 5:"20-24", 6:"25-29", 7:"30-34", 8:"35-39", 9:"40-44", 10:"45-49", 11:"50-54", 12:"55-59", 13:"60-64", 14:"65-69", 15:"70-74", 16:"75-79", 17:"80-84", 18:"85-89", 19:"90-94", 20:"95-99",21:"100+"}}),
                        ("GE",       {"name": "GE",                     "desc": "Giorno di decesso (formato variabile: MeseMeseGiornoGiorno).",
                                      "type": __type2name(str),         "values": None}),
                        ("M_15",       {"name": "MASCHI_15",                     "desc": "numero di decessi maschili nel 2015.",
                                      "type": __type2name(int),         "values": None}),
                        ("M_16",       {"name": "MASCHI_16",                     "desc": "numero di decessi maschili nel 2016.",
                                      "type": __type2name(int),         "values": None}),
                        ("M_17",       {"name": "MASCHI_17",                     "desc": "numero di decessi maschili nel 2017.",
                                      "type": __type2name(int),         "values": None}),
                        ("M_18",       {"name": "MASCHI_18",                     "desc": "numero di decessi maschili nel 2018.",
                                      "type": __type2name(int),         "values": None}),
                        ("M_19",       {"name": "MASCHI_19",                     "desc": "numero di decessi maschili nel 2019.",
                                      "type": __type2name(int),         "values": None}),
                        ("M_20",       {"name": "MASCHI_20",                     "desc": "numero di decessi maschili nel 2020.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_15",       {"name": "FEMMINE_15",                     "desc": "numero di decessi femminili nel 2015.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_16",       {"name": "FEMMINE_16",                     "desc": "numero di decessi femminili nel 2016.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_17",       {"name": "FEMMINE_17",                     "desc": "numero di decessi femminili nel 2017.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_18",       {"name": "FEMMINE_18",                     "desc": "numero di decessi femminili nel 2018.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_19",       {"name": "FEMMINE_19",                     "desc": "numero di decessi femminili nel 2019.",
                                      "type": __type2name(int),         "values": None}),
                        ("F_20",       {"name": "FEMMINE_20",                     "desc": "numero di decessi femminili nel 2020.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_15",       {"name": "TOTALE_15",                     "desc": "numero di decessi totali nel 2015.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_16",       {"name": "TOTALE_16",                     "desc": "numero di decessi totali nel 2016.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_17",       {"name": "TOTALE_17",                     "desc": "numero di decessi totali nel 2017.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_18",       {"name": "TOTALE_18",                     "desc": "numero di decessi totali nel 2018.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_19",       {"name": "TOTALE_19",                     "desc": "numero di decessi totali nel 2019.",
                                      "type": __type2name(int),         "values": None}),
                        ("T_20",       {"name": "TOTALE_20",                     "desc": "numero di decessi totali nel 2020.",
                                      "type": __type2name(int),         "values": None})
                        ]),
                    "nan": 9999
}

Actually, at this stage, we are interested in the data source only:

In [95]:
SOURCE = METADATA['source']
print('Input source file: %s' % SOURCE)

PROTOCOLS = ['http', 'https', 'ftp'] 
if any([SOURCE.startswith(p) for p in PROTOCOLS]):
    try:
        response = requests.get(SOURCE)
        response.raise_for_status()
    except (requests.URLRequired,requests.HTTPError,requests.RequestException):
        print("NO source file found online... abort!")
    else:
        source = io.BytesIO(response.content)
        print("Source file found online... proceed")
elif os.path.exists(SOURCE):
    source = SOURCE
    print("Source file found on the disk... proceed")
else:
    print("NO input source file found on the disk... abort!")

Input source file: https://www.istat.it/it/files/2020/03/comune-giorno.zip
Source file found online... proceed


In [96]:
FILE = METADATA['file']
print('Input data file: %s' % FILE)

ENC = METADATA['enc']
SEP = METADATA['sep']

if FILE is None:
    file = source
if any([SOURCE.endswith(z) for z in ('zip','gzip','gz')]):
    try:
        assert zipfile.is_zipfile(source)
    except:
        print("Wrongly formatted input source file... abort!")
    with zipfile.ZipFile(source) as zf:
        namelist = zf.namelist()
        try:
            assert FILE in namelist
        except:
            try:
                _namelist = [os.path.basename(n) for n in namelist]
                assert FILE in _namelist
            except:
                print("Data not found in source file... abort!")
            else:
                file = namelist[_namelist.index(FILE)]
        else:
            file = FILE
        try:
            # data = zf.read(file)
            with zf.open(file) as zd:
                data = pd.read_csv(zd, encoding=ENC)        
        except:
            print("Data %s cannot be read in source file... abort!" % file)
        else:
            print("Data %s read in source file" % file)

Input data file: comune_giorno.csv
Data comune-giorno/comune_giorno.csv read in source file


Let's have a first look at the data:

In [97]:
data.head(5)

,REG,PROV,NOME_REGIONE,NOME_PROVINCIA,NOME_COMUNE,COD_PROVCOM,CL_ETA,GE,MASCHI_15,MASCHI_16,...,FEMMINE_17,FEMMINE_18,FEMMINE_19,FEMMINE_20,TOTALE_15,TOTALE_16,TOTALE_17,TOTALE_18,TOTALE_19,TOTALE_20
0,1,1,Piemonte,Torino,Agliè,1001,17,102,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1,1,Piemonte,Torino,Agliè,1001,18,104,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,1,Piemonte,Torino,Agliè,1001,18,105,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,Piemonte,Torino,Agliè,1001,17,106,1,0,...,0,0,0,0,1,0,0,0,0,0
4,1,1,Piemonte,Torino,Agliè,1001,18,106,0,0,...,0,0,0,0,0,0,0,1,0,0


What are the fields? It should be consistent with our metadata...

In [98]:
try:
    assert set(list(data.columns)) == set([v['name'] for v in METADATA['index'].values()])
except:
    print("Mismatch between metadata and actual columns in the dataset")
print('Fields of the data: %s' % list(data.columns))

Fields of the data: ['REG', 'PROV', 'NOME_REGIONE', 'NOME_PROVINCIA', 'NOME_COMUNE', 'COD_PROVCOM', 'CL_ETA', 'GE', 'MASCHI_15', 'MASCHI_16', 'MASCHI_17', 'MASCHI_18', 'MASCHI_19', 'MASCHI_20', 'FEMMINE_15', 'FEMMINE_16', 'FEMMINE_17', 'FEMMINE_18', 'FEMMINE_19', 'FEMMINE_20', 'TOTALE_15', 'TOTALE_16', 'TOTALE_17', 'TOTALE_18', 'TOTALE_19', 'TOTALE_20']


How many records?

In [99]:
print('#Records: %s - #Fields: %s' % data.shape)

#Records: 841031 - #Fields: 26


Let's get rid of those records with no data, _i.e._ with 9999 in the `T_20` column

In [101]:
NAN = METADATA['nan']
it_20 = METADATA['index']['T_20']['name']
# data = data[data[it_20] != NAN]
data.drop(data.loc[data[it_20]==NAN].index, inplace=True)
print('#Records: %s - #Fields: %s' % data.shape)

#Records: 198076 - #Fields: 26


Let's retrieve for the set of unique "comune" for further geocoding:

In [105]:
icomune = METADATA['index']['COMUNE']['name']
comune = data[icomune].unique()
print('List of 50 first commune in the dataset: %s' % comune[:50])

List of 50 first commune in the dataset: ['Agliè' 'Almese' 'Avigliana' 'Banchette' 'Beinasco' 'Bosconero' 'Bruino'
 'Buttigliera Alta' 'Carmagnola' 'Castellamonte' 'Condove' 'Frossasco'
 'Gassino Torinese' 'Grugliasco' 'Ivrea' 'Lanzo Torinese'
 'Luserna San Giovanni' 'Mathi' 'None' 'Orbassano' 'Pancalieri'
 'Piossasco' 'Piscina' 'Poirino' 'Riva presso Chieri' 'Rivara'
 'Rivarolo Canavese' 'Sangano' 'San Giorgio Canavese'
 'San Giusto Canavese' 'San Mauro Torinese' 'San Raffaele Cimena'
 'San Secondo di Pinerolo' 'Strambino' 'Susa' 'Trofarello' 'Valperga'
 'Verolengo' 'Villar Perosa' 'Villastellone' 'Volpiano' 'Borgosesia'
 'Buronzo' 'Caresanablot' 'Gattinara' 'Roasio' 'Saluggia' 'Santhià'
 'Trino' 'Varallo']


Let's join this with the table of geolocated "comunes"

### Data analysis